In [1]:
import pandas as pd

# csv 파일 읽어오기
customer_master = pd.read_csv("excelFiles/1장/customer_master.csv")
# 읽어들인 파일 처음 5행 표시
customer_master.head()
item_master = pd.read_csv("excelFiles/1장/item_master.csv")
item_master.head()
transaction_1 = pd.read_csv("excelFiles/1장/transaction_1.csv")
transaction_1.head()
transaction_detail_1 = pd.read_csv("excelFiles/1장/transaction_detail_1.csv")
transaction_detail_1.head()
# transaction1 , transaction2 세로로 합친 후 transaction 데이터 프레임 변수에 저장
# ignore_index = True --> 기존 Index 무시

transaction_2 = pd.read_csv("excelFiles/1장/transaction_2.csv")
transaction = pd.concat([transaction_1, transaction_2], ignore_index=True)
transaction.head()
"""
pd.concat(objs,  # Series, DataFrame, Panel object
             axis=0,  # 0: 위+아래로 합치기, 1: 왼쪽+오른쪽으로 합치기
             join='outer', # 'outer': 합집합(union), 'inner': 교집합(intersection)
             ignore_index=False,  # False: 기존 index 유지, True: 기존 index 무시
             keys=None, # 계층적 index 사용하려면 keys 튜플 입력
             levels=None, 
             names=None, # index의 이름 부여하려면 names 튜플 입력
             verify_integrity=False, # True: index 중복 확인
             copy=True) # 복사
"""
# 합쳐진 데이터프레임 길이 확인

print(len(transaction_1))
print(len(transaction_2))
print(len(transaction))
# transaction 데이터 결합

transaction_detail_2 = pd.read_csv("excelFiles/1장/transaction_detail_2.csv")
transaction_detail = pd.concat([transaction_detail_1, transaction_detail_2], ignore_index=True)
transaction_detail.head()

'''

pd.merge(left, right, on = '기준열', how = '조인방식') : 기준열 이름이 같을때
pd.merge(left, right, left_on = '왼쪽 열', right_on = '오른쪽 열', how = '조인방식') : 기준열 이름이 다를떄

left : 왼쪽 데이터프레임
right : 오른쪽 데이터프레임
on : (두 데이터프레임의 기준열 이름이 같을 때) 기준열
how : 조인 방식 {'left', 'right', 'inner', 'outer'} 기본값은 'inner'
 
left_on : 기준열 이름이 다를 때, 왼쪽 기준열
right_on : 기준열 이름이 다를 때, 오른쪽 기준열
'''
join_data = pd.merge(transaction_detail, transaction[["transaction_id", "payment_date", "customer_id"]],
                     on="transaction_id", how="left")
join_data.head()
print(len(transaction_detail))
print(len(transaction))
print(len(join_data))

# 데이터 개수 변경 x
join_data = pd.merge(join_data, customer_master, on="customer_id", how="left")
join_data = pd.merge(join_data, item_master, on="item_id", how="left")
join_data.head()
join_data["price"] = join_data["quantity"] * join_data["item_price"]
join_data[["quantity", "item_price", "price"]].head()
# join_data 가장 마지막 열에 추가
join_data.head()
'''
데이터의 검산
join_data 의 price 와
transaction 의 price를 비교

sum() : 객체의 열이나 행의 총 합을 구함
'''

print(join_data["price"].sum())
print(join_data["price"].sum())
# True or False 로 반환하여 검산

join_data["price"].sum() == transaction["price"].sum()
'''
isnull() : 결손치를 True or False로 반환
isnull().sum() : True의 개수를 합하여 출력
'''

join_data.isnull().sum()
'''
describe() : 통계량 출력
count : 개수
mean : 평균 
std : 표준편차
min : 최솟값
25%, 50%, 75% : 사분위수 
max : 최댓값
'''

join_data.describe()
# 기간 데이터의 최대 최소 확인

print(join_data["payment_date"].min())
print(join_data["payment_date"].max())
'''
payment_date 에서 연월을 추출해서 새롭게 칼럼을 구성
dtypes : 칼럼마다 데이터형 확인 할 수 있다.
payment_date : object형 (=string)
'''

join_data.dtypes
'''
payment_date를 datetime형으로 변환
datetime의 dt를 사용해 년,월의 추출 가능
strftime : 날짜와 시간을 문자열로 표현
strptime : 문자열을 날짜와 시간으로 표현
'''

join_data["payment_date"] = pd.to_datetime(join_data["payment_date"])
join_data["payment_month"] = join_data["payment_date"].dt.strftime("%Y%m")
join_data[["payment_date", "payment_month"]].head()
'''
책에서는 join_data.groupby("payment_month").sum()["price"]라고 나와있음
'''
join_data.groupby("payment_month")["price"].sum()
join_data.groupby(["payment_month", "item_name"])[["price", "quantity"]].sum()
'''
행과 칼럼 지정
예제에서는 
행(index) : 상품명, 열(columns) : 월
aggfunc : 계산을 어떻게 할지 (ex. sum, )
'''
pd.pivot_table(join_data, index='item_name', columns='payment_month', values=['price', 'quantity'], aggfunc='sum')
graph_data = pd.pivot_table(join_data, index='payment_month', columns='item_name', values='price', aggfunc='sum')
graph_data.head()
import matplotlib.pyplot as plt

plt.plot(list(graph_data.index), graph_data["PC-A"], label='PC-A')
plt.plot(list(graph_data.index), graph_data["PC-B"], label='PC-B')
plt.plot(list(graph_data.index), graph_data["PC-C"], label='PC-C')
plt.plot(list(graph_data.index), graph_data["PC-D"], label='PC-D')
plt.plot(list(graph_data.index), graph_data["PC-E"], label='PC-E')

plt.legend()

FileNotFoundError: [Errno 2] No such file or directory: 'excelFiles/1장/customer_master.csv'